In [80]:
#Importamos las librerias necesarias para crear la red neuronal
import pandas
import numpy as np
from sklearn import model_selection, preprocessing
from tensorflow import keras
import tensorflow as tf
#Leemos el csv
evaluation = pandas.read_csv('turkiye-student-evaluation_R_Specific.csv', header=0)

In [81]:
#Mostramos las 10 primeras lineas para ver que se ha leido bien
evaluation.head(10)

,instr,class,nb.repeat,attendance,difficulty,Q1,Q2,Q3,Q4,Q5,...,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28
1,1,2,1,0,4,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,1,2,1,1,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3,1,2,1,2,4,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
4,1,2,1,1,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
5,1,2,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,1,2,1,3,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
7,1,2,1,1,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
8,1,2,1,1,3,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
9,1,2,1,1,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
10,1,2,1,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [82]:
#Seleccionamos los atributos 
atributos = evaluation.loc[:, 'class':'Q27']
atributos = atributos.to_numpy()
scaler = preprocessing.MinMaxScaler()
atributos = scaler.fit_transform(atributos)
print(atributos[5818,:])

[1.   0.   0.25 0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.  ]


In [83]:
#Seleccionamos el objetivo
objetivo = evaluation['Q28']
objetivo = evaluation['Q28'].to_numpy(dtype=np.float32)
print(objetivo)

[3. 3. 5. ... 5. 1. 1.]


In [84]:
#Separamos los datos que se usaran para entrenar y los que se usarán para evaluar los modelos
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
    atributos, objetivo, test_size=.15)

In [85]:
normalizador = keras.layers.Normalization()#Definimos un normalizador
normalizador.adapt(atributos_entrenamiento)
red_evaluation = keras.Sequential()#Creamos la red neuronal
red_evaluation.add(keras.layers.Input(shape=(31,)))#Definimos la capa de entrada con una neurona por cada atributo
red_evaluation.add(normalizador)#Aplicamos el normalizador
red_evaluation.add(keras.layers.Dense(70, activation='relu'))#Definimos una capa con 70 neuronas y función de activación relu
red_evaluation.add(keras.layers.Dense(70, activation='sigmoid'))#Definimos una capa de 70 neuronas y función de activación sigmoide
red_evaluation.add(keras.layers.Dense(1, activation='linear')) #Defnimios una capa de salida con función de activación lienal ya que los valores esperados son números lineales entre 0 y 5

In [86]:
red_evaluation.summary()#Mostramos un resumen de la red neuronal

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_2 (Normalizat  (None, 31)               63        
 ion)                                                            
                                                                 
 dense_6 (Dense)             (None, 70)                2240      
                                                                 
 dense_7 (Dense)             (None, 70)                4970      
                                                                 
 dense_8 (Dense)             (None, 1)                 71        
                                                                 
Total params: 7,344
Trainable params: 7,281
Non-trainable params: 63
_________________________________________________________________


In [87]:

red_evaluation.compile(optimizer='SGD', loss='mean_squared_error', metrics=['accuracy', 'mae'])#Compilamos la red neuronal definiendo el optimizaodr, la función de perdida y dos metricas
red_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=256, epochs=10)#Entrenamos la red neuronal con 10 epocas y lotes de 256 elementos

Epoch 1/10
20/20 [==============================] - 1s 2ms/step - loss: 2.7755 - accuracy: 0.1326 - mae: 1.1793 
Epoch 2/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4213 - accuracy: 0.1405 - mae: 0.4605
Epoch 3/10
20/20 [==============================] - 0s 2ms/step - loss: 0.3195 - accuracy: 0.1405 - mae: 0.3584
Epoch 4/10
20/20 [==============================] - 0s 2ms/step - loss: 0.3027 - accuracy: 0.1405 - mae: 0.3329
Epoch 5/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2951 - accuracy: 0.1405 - mae: 0.3228
Epoch 6/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2895 - accuracy: 0.1405 - mae: 0.3217
Epoch 7/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2846 - accuracy: 0.1405 - mae: 0.3128
Epoch 8/10
20/20 [==============================] - 0s 3ms/step - loss: 0.2806 - accuracy: 0.1405 - mae: 0.3104
Epoch 9/10
20/20 [==============================] - 0s 3ms/step - loss: 0.2766 - accuracy: 0.1405 - mae

In [88]:
#Definimos cuanta perdida queremos tener y ejecutamos el siguiente bucle para entrenar la red hasta conseguir una perdida menor a la definida
#Para este conjunto de datos esto practicamente no es necesario pero así nos aseguramos de cometer un error muy pequeño
perdObj = 1.0
x=red_evaluation.evaluate(atributos_prueba, objetivo_prueba)
while x[0]>perdObj:
    red_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=256, epochs=10)
    x=red_evaluation.evaluate(atributos_prueba, objetivo_prueba)
    
print(x[0])

28/28 [==============================] - 0s 2ms/step - loss: 0.2403 - accuracy: 0.1340 - mae: 0.2808
0.24028357863426208


In [89]:
#Ahora que ya hemos conseguido una perdida menor a la que queriamos, evaluamos para volver a comprobar que la hemos conseguido
red_evaluation.evaluate(atributos_prueba, objetivo_prueba)

28/28 [==============================] - 0s 1ms/step - loss: 0.2403 - accuracy: 0.1340 - mae: 0.2808


[0.24028357863426208, 0.1340206116437912, 0.28082191944122314]

In [90]:
#Ahora mismo comprobaremos manualmente si las predicciones de la red se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
X=atributos[5809]
predicciones = red_evaluation.predict(X)
print('Esperado 4:',predicciones)
X=atributos[5810]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5811]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5812]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5813]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5814]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5815]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5816]
predicciones = red_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos[5817]
predicciones = red_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos[5818]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5819]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)

1/1 [==============================] - 0s 83ms/step
Esperado 4: [[4.0871906]]
1/1 [==============================] - 0s 30ms/step
Esperado 3: [[3.0435562]]
1/1 [==============================] - 0s 25ms/step
Esperado 3: [[3.1551712]]
1/1 [==============================] - 0s 28ms/step
Esperado 1: [[1.0064591]]
1/1 [==============================] - 0s 27ms/step
Esperado 3: [[3.0156977]]
1/1 [==============================] - 0s 33ms/step
Esperado 1: [[1.1983334]]
1/1 [==============================] - 0s 33ms/step
Esperado 1: [[1.0064591]]
1/1 [==============================] - 0s 35ms/step
Esperado 5: [[4.730139]]
1/1 [==============================] - 0s 34ms/step
Esperado 5: [[4.845992]]
1/1 [==============================] - 0s 31ms/step
Esperado 1: [[1.0261306]]
1/1 [==============================] - 0s 37ms/step
Esperado 1: [[1.0261306]]


In [91]:
#Ahora trabajaremos con el otro modelo que será un random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
forest_evaluation = RandomForestRegressor() #Creamos el modelo
forest_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento)#Lo entrenamos
evaluaciones = forest_evaluation.predict(atributos_prueba)#Evaluamos
mse = mean_squared_error(objetivo_prueba, evaluaciones)#Calculamos la perdida
print("Error cuadrático medio (MSE) del modelo:", mse)
#Como este modelo suele tener una perdida muy pequeña tras entrenarlo una vez, no hacemos nada más para mejorarlo

Error cuadrático medio (MSE) del modelo: 0.24148652713073981


In [92]:
#Ahora mismo comprobaremos manualmente si las predicciones del random forest se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
atributos2 = atributos[:, np.newaxis]
X=atributos2[5809]
predicciones = forest_evaluation.predict(X)
print('Esperado 4:',predicciones)
X=atributos2[5810]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5811]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5812]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5813]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5814]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5815]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5816]
predicciones = forest_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos2[5817]
predicciones = forest_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos2[5818]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5819]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)

Esperado 4: [4.]
Esperado 3: [3.]
Esperado 3: [3.]
Esperado 1: [1.]
Esperado 3: [3.]
Esperado 1: [1.2]
Esperado 1: [1.]
Esperado 5: [4.96]
Esperado 5: [5.]
Esperado 1: [1.]
Esperado 1: [1.]


In [93]:
#Importamos las librerias necesarias para crear otra red neuronal
#Volemos a importar por si se quiere ejecutar este fragmento sin ejecutar el anterior
import pandas
import numpy as np
from sklearn import model_selection, preprocessing
from tensorflow import keras
import tensorflow as tf

#Leemos el csv
#bikes = pandas.read_csv('hour.csv', header=0)
bikes = pandas.read_csv('hour2.csv',header = 0)


In [94]:
#Mostramos las 10 primeras lineas para ver que se ha leido bién
bikes.head(10)

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
5,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
6,1,0,1,6,0,6,0,1,0.22,0.2727,0.80,0.0000,2,0,2
7,1,0,1,7,0,6,0,1,0.20,0.2576,0.86,0.0000,1,2,3
8,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0.0000,1,7,8
9,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0.0000,8,6,14


In [95]:
#Seleccionamos los atributos y los nomalizamos
attr = bikes.loc[:, 'season':'registered']
attr = attr.to_numpy()
scaler = preprocessing.MinMaxScaler()
attr = scaler.fit_transform(attr)

print(attr)

[[0.         0.         0.         ... 0.         0.00817439 0.01467269]
 [0.         0.         0.         ... 0.         0.02179837 0.03611738]
 [0.         0.         0.         ... 0.         0.01362398 0.03047404]
 ...
 [0.         1.         1.         ... 0.19301751 0.01907357 0.09367946]
 [0.         1.         1.         ... 0.15786999 0.03542234 0.05417607]
 [0.         1.         1.         ... 0.15786999 0.03269755 0.04176072]]


In [96]:
#Seleccionamos los objetivos
obj = bikes['cnt']
obj = bikes['cnt'].to_numpy(dtype=np.float32)
print(obj)

[16. 40. 32. ... 90. 61. 49.]


In [97]:
#Cogemos un 85% de los datos para entrenar y un 15% de los datos para evaluar el modelo
(attr_entrenamiento, attr_prueba,
 obj_entrenamiento, obj_prueba) = model_selection.train_test_split(
    attr, obj, test_size=.10)

In [98]:
normalizador = keras.layers.Normalization()#Definimos un normalizador
normalizador.adapt(attr_entrenamiento)
red_bikes = keras.Sequential()#Creamos la red neuronal
red_bikes.add(keras.layers.Input(shape=(14,))) #Añadimos la capa de entrada con 14 neuronas, una por cada atributo
red_bikes.add(normalizador)#Aplicamos un normalizador
red_bikes.add(keras.layers.Dense(100, activation='sigmoid'))#Añadimos una capa intermedia de 100 neuronas con función de activación sigmoide
red_bikes.add(keras.layers.Dense(100, activation='sigmoid'))#Añadimos una capa intermedia de 100 neuronas con función de activación sigmoide
red_bikes.add(keras.layers.Dense(1, activation='linear'))#Necesitamos una capa de salida linear ya que los valores objetivos son lineales

In [99]:
#Vemos un resumen de la red
red_bikes.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_3 (Normalizat  (None, 14)               29        
 ion)                                                            
                                                                 
 dense_9 (Dense)             (None, 100)               1500      
                                                                 
 dense_10 (Dense)            (None, 100)               10100     
                                                                 
 dense_11 (Dense)            (None, 1)                 101       
                                                                 
Total params: 11,730
Trainable params: 11,701
Non-trainable params: 29
_________________________________________________________________


In [100]:
#Compilamos y entrenamos la red neuronal, buscamos minimizar la función de perdida
red_bikes.compile(optimizer='SGD', loss='mean_squared_error', metrics=['accuracy', 'mae'])
red_bikes.fit(attr_entrenamiento, obj_entrenamiento,
                batch_size=256, epochs=10)

Epoch 1/10
62/62 [==============================] - 1s 2ms/step - loss: 13533.7861 - accuracy: 0.0087 - mae: 80.8802
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 2811.9653 - accuracy: 0.0088 - mae: 39.7757
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 1598.3772 - accuracy: 0.0088 - mae: 31.1913
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 977.5739 - accuracy: 0.0086 - mae: 21.5425
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 796.2078 - accuracy: 0.0081 - mae: 21.0913
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 407.9771 - accuracy: 0.0088 - mae: 15.8295
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 419.1377 - accuracy: 0.0084 - mae: 15.6306
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 299.3969 - accuracy: 0.0088 - mae: 13.9114
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 235.

In [101]:
#Definimos cuanta perdida queremos tener y ejecutamos el siguiente bucle para entrenar la red hasta conseguir una perdida menor a la definida
perdidaObj = 1.0
x=red_bikes.evaluate(attr_prueba, obj_prueba)
while x[0]>perdidaObj:
    red_bikes.fit(attr_entrenamiento, obj_entrenamiento,
                batch_size=256, epochs=10)
    x=red_bikes.evaluate(attr_prueba, obj_prueba)
    
#La forma más efectiva que hemos encontrado para mejorar nuestra red ha sido la de hacer este bulce ya que es la unica forma de asegurarnos que la red obtendrá la precisión que busquemos
print(x[0])

55/55 [==============================] - 0s 2ms/step - loss: 58.0273 - accuracy: 0.0121 - mae: 5.3620
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 167.7412 - accuracy: 0.0084 - mae: 9.6977
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 158.6113 - accuracy: 0.0088 - mae: 10.0131
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 187.0991 - accuracy: 0.0088 - mae: 11.1683
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 151.4776 - accuracy: 0.0088 - mae: 9.8628
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 172.1601 - accuracy: 0.0088 - mae: 10.6888
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 137.1284 - accuracy: 0.0085 - mae: 8.4927
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 88.5596 - accuracy: 0.0087 - mae: 7.2088
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 176.8708 - accuracy: 0.008

Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 7.4419 - accuracy: 0.0088 - mae: 2.0579
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 43.9864 - accuracy: 0.0088 - mae: 5.6472
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 25.8900 - accuracy: 0.0088 - mae: 4.2019
Epoch 10/10
55/55 [==============================] - 0s 1ms/step - loss: 79.1642 - accuracy: 0.0121 - mae: 6.4509
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 56.5566 - accuracy: 0.0086 - mae: 6.2378
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 20.7937 - accuracy: 0.0088 - mae: 3.5298
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 20.2237 - accuracy: 0.0088 - mae: 3.3820
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 28.1174 - accuracy: 0.0081 - mae: 3.7413
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 68.4894 - accuracy: 0.00

55/55 [==============================] - 0s 1ms/step - loss: 3.0189 - accuracy: 0.0121 - mae: 1.3476
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 8.5186 - accuracy: 0.0088 - mae: 2.4382
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 13.7007 - accuracy: 0.0088 - mae: 3.2245
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 8.7766 - accuracy: 0.0088 - mae: 2.4867
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 12.8430 - accuracy: 0.0088 - mae: 3.0735
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 11.6392 - accuracy: 0.0088 - mae: 2.9415
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 5.5348 - accuracy: 0.0088 - mae: 1.9303
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 13.6327 - accuracy: 0.0088 - mae: 3.2433
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 12.2667 - accuracy: 0.0088 - mae: 3.046

62/62 [==============================] - 0s 2ms/step - loss: 9.7682 - accuracy: 0.0088 - mae: 2.6468
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 4.8792 - accuracy: 0.0088 - mae: 1.7360
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 25.3147 - accuracy: 0.0082 - mae: 4.4323
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 3.4859 - accuracy: 0.0088 - mae: 1.4141
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 2.8319 - accuracy: 0.0088 - mae: 1.3810
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 8.0242 - accuracy: 0.0088 - mae: 2.4701
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 12.5598 - accuracy: 0.0088 - mae: 3.2071
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 2.4389 - accuracy: 0.0088 - mae: 1.2463
Epoch 10/10
55/55 [==============================] - 0s 1ms/step - loss: 11.5928 - accuracy: 0.0121 - mae: 2.8330

62/62 [==============================] - 0s 2ms/step - loss: 15.4369 - accuracy: 0.0088 - mae: 3.5212
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 8.5023 - accuracy: 0.0088 - mae: 2.3624
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 1.5271 - accuracy: 0.0088 - mae: 0.9916
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 4.4699 - accuracy: 0.0088 - mae: 1.7938
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 6.1918 - accuracy: 0.0088 - mae: 2.2227
Epoch 10/10
55/55 [==============================] - 0s 1ms/step - loss: 4.6380 - accuracy: 0.0121 - mae: 1.7444
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 12.4155 - accuracy: 0.0088 - mae: 3.2080
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 6.3957 - accuracy: 0.0088 - mae: 2.2356
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 15.6032 - accuracy: 0.0088 - mae: 3.5361

62/62 [==============================] - 0s 2ms/step - loss: 6.7108 - accuracy: 0.0088 - mae: 2.2523
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 2.2110 - accuracy: 0.0088 - mae: 1.2341
Epoch 10/10
55/55 [==============================] - 0s 1ms/step - loss: 1.1527 - accuracy: 0.0121 - mae: 0.8515
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 5.0383 - accuracy: 0.0088 - mae: 1.8820
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 9.1197 - accuracy: 0.0088 - mae: 2.7545
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 2.8027 - accuracy: 0.0088 - mae: 1.2483
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 3.4526 - accuracy: 0.0088 - mae: 1.5954
Epoch 5/10
62/62 [==============================] - 0s 2ms/step - loss: 7.7232 - accuracy: 0.0088 - mae: 2.5425
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 9.1835 - accuracy: 0.0088 - mae: 2.7318
Ep

In [102]:
#Ahora que ya hemos conseguido una perdida menor a la que queriamos, evaluamos para volver a comprobar que la hemos conseguido
red_bikes.evaluate(attr_prueba, obj_prueba)

55/55 [==============================] - 0s 1ms/step - loss: 0.9382 - accuracy: 0.0121 - mae: 0.7710


[0.9381737112998962, 0.01208285428583622, 0.7709532976150513]

In [103]:
#Ahora mismo comprobaremos manualmente si las predicciones de la red se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado

X=attr[17368]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 203:',predicciones)
X=attr[17369]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 247:',predicciones)
X=attr[17370]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 315:',predicciones)
X=attr[17371]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 214:',predicciones)
X=attr[17372]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 164:',predicciones)
X=attr[17373]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 122:',predicciones)
X=attr[17374]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 119:',predicciones)
X=attr[17375]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 89:',predicciones)
X=attr[17376]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 90:',predicciones)
X=attr[17377]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 61:',predicciones)
X=attr[17378]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 49:',predicciones)

Esperado 203: [[201.52798]]
Esperado 247: [[247.29338]]
Esperado 315: [[315.31146]]
Esperado 214: [[214.59116]]
Esperado 164: [[162.66823]]
Esperado 122: [[121.300385]]
Esperado 119: [[119.077126]]
Esperado 89: [[88.58235]]
Esperado 90: [[90.05068]]
Esperado 61: [[61.305023]]
Esperado 49: [[47.611996]]


In [104]:
#Ahora trabajaremos con el otro modelo que será un random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
forest_bikes = RandomForestRegressor()#Creamos el randomForest
forest_bikes.fit(attr_entrenamiento, obj_entrenamiento)#Lo entrenamos
evaluaciones = forest_bikes.predict(attr_prueba)#Evaluamos el modelo
mse = mean_squared_error(obj_prueba, evaluaciones)#Calculamos la perdida
print("Error cuadrático medio (MSE) del modelo:", mse)

Error cuadrático medio (MSE) del modelo: 4.465658745684692


In [105]:
#Ahora mismo comprobaremos manualmente si las predicciones del random forest se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
attr2 = attr[:, np.newaxis]
X=attr2[17368]
predicciones = forest_bikes.predict(X)
print('Esperado 203:',predicciones)
X=attr2[17369]
predicciones = forest_bikes.predict(X)
print('Esperado 247:',predicciones)
X=attr2[17370]
predicciones = forest_bikes.predict(X)
print('Esperado 315:',predicciones)
X=attr2[17371]
predicciones = forest_bikes.predict(X)
print('Esperado 214:',predicciones)
X=attr2[17372]
predicciones = forest_bikes.predict(X)
print('Esperado 164:',predicciones)
X=attr2[17373]
predicciones = forest_bikes.predict(X)
print('Esperado 122:',predicciones)
X=attr2[17374]
predicciones = forest_bikes.predict(X)
print('Esperado 119:',predicciones)
X=attr2[17375]
predicciones = forest_bikes.predict(X)
print('Esperado 89:',predicciones)
X=attr2[17376]
predicciones = forest_bikes.predict(X)
print('Esperado 90:',predicciones)
X=attr2[17377]
predicciones = forest_bikes.predict(X)
print('Esperado 61:',predicciones)
X=attr2[17378]
predicciones = forest_bikes.predict(X)
print('Esperado 49:',predicciones)

Esperado 203: [203.]
Esperado 247: [246.94]
Esperado 315: [313.61]
Esperado 214: [215.02]
Esperado 164: [163.98]
Esperado 122: [121.91]
Esperado 119: [118.94]
Esperado 89: [88.95]
Esperado 90: [89.92]
Esperado 61: [61.09]
Esperado 49: [49.06]


In [106]:
#Metodo LIME
import random
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPClassifier, MLPRegressor



"""
Partiremos de este seudocódigo para implementar el metodo
N es el n´umero de permutaciones a realizar
f es el modelo a explicar
X′ ← {} muestras perturbadas
R ← {} representaciones
W ← {} las distancias entre la muestra x y sus perturbaciones
for 1
to
N do
Selecciona k atributos aleatoriamente
x′ ← una perturbaci´on de x donde se perturban los k atributos anteriores.
w ← la distancia entre x y x′
r ← la representaci´on de x′ respecto a x
X′ ← X′ ∪ x′
R ← R ∪ r
W ← R ∪ w
end for
Y ′ ← f(X′) las predicciones de las perturbaciones
G ← modelo ridge entrenado con R para predecir Y ′ y ponderando cada
muestra con W
return los par´ametros de G

"""

def explain_model(f, x, N,M):
    #f es el modelo a explicar
    #X es una lista de ejemplos a los que se aplicará LIME
    #N es el número de permutaciones a realizar
    #M contiene todos los ejemplos
    #Si queremos que el algoritmo cumpla la metrica identidad necesitamos fijar semillas para la aleatoriedad
    random.seed(11)
    Xi = [] #Aqui guardaremos las muestras perturbadas
    R = []  #Aqui guardaremos las representaciones
    W = []  #Aqui guradaremos las distancias
    for i in range(N):
        k = random.randint(1, len(x)) #Escojo un número aletario que representa el número de los atributos a seleccionar
        perturbed_x = x.copy() #copio la muestra original
        for j in range(k): 
            perturbed_attr = random.randint(0,len(x)-1)#Voy escojiento los atributos que perturbare de forma aleatoria
            mx = abs(max([a[perturbed_attr] for a in M])) #Para el atributo aletorio seleccionado calculo el valor máximo
            mn = abs(min([a[perturbed_attr] for a in M])) #Para el atributo aletorio seleccionado calculo el valor mínimo
            perturbed_x[perturbed_attr] = random.uniform(mn, mx) # pertubo el atributo con un valor aleatorio ente mn y mx para que el valor esté acotado
        w = abs(sum([x[attr] - perturbed_x[attr] for attr in range(0,len(x)-1)])) #Calculo la distancia entre el original y el perturbado
        r = [0  if perturbed_x[attr] == x[attr] else 1 for attr in range(0,len(x)-1)] #Calculo la representación del perturbado respecto al original utilizando un operador ternario
        Xi.append(perturbed_x) #Acumulo en la lista los perturbados
        R.append(r)#Acumulamos en la lista de representaciones
        W.append(w)#Añadimos las distancias a la lista
    Y_perturbed = []#Aqui guardaremos las predicciones de las perturbaciones
    for i in range(len(Xi)):
        y = 1
            #En los siguiente if y else comprobamos el tipo de modelo que tenemos ya que dependiendo del tipo necesita recibir los atributos de una forma u otra
        if isinstance(f, keras.Sequential):#Comprueba si el modelo es una red neuronal
            xi = Xi[i]
            y = f.predict(xi,verbose=0)#Hacemos la prediccion con verbose=0 para no cargar la salida de lineas generadas por keras
        else: #Si no, es un randomForest
            xi = Xi[i]
            array = np.array(xi)
            xi = array.reshape(1, -1)
            y = f.predict(xi)#RandomForest no tiene verbose porque no genera nada
                
            
        Y_perturbed.append(y) #Aplico el modelo f a los ejemplos perturbados
            
    Y_perturbed = np.squeeze(Y_perturbed) #Por alguna razón aparece el error Found array with dim 3. Estimator expected <= 2" y lo arreglamos con esta linea que quita una dimensión al array
    G = Ridge()
    G.fit(R, Y_perturbed, sample_weight=W)
    return G.coef_#Devolvemos los parametros


In [107]:
#Hacemos varias pruebas para comprobar que funciona el metodo LIME
atributivos = atributos[5809]

explain_model(red_evaluation,atributivos,10,atributos)

array([-0.12209265,  0.01907802, -0.01072902, -0.06241706, -0.02617428,
        0.02137444, -0.08280167, -0.06579414, -0.04063194, -0.00866057,
        0.01660673, -0.02056638, -0.07760425,  0.01122755, -0.02195677,
       -0.08298343, -0.04912087, -0.01072922, -0.07058429,  0.07745163,
       -0.00535004, -0.06034842, -0.03129559,  0.106972  , -0.08280167,
        0.01221738,  0.00380703,  0.00080806, -0.03947047, -0.02006989])

In [108]:
#Hacemos varias pruebas para comprobar que funciona el metodo LIME
atributivos = atributos[5809]

explain_model(red_evaluation,atributivos,10,atributos)

array([-0.12209265,  0.01907802, -0.01072902, -0.06241706, -0.02617428,
        0.02137444, -0.08280167, -0.06579414, -0.04063194, -0.00866057,
        0.01660673, -0.02056638, -0.07760425,  0.01122755, -0.02195677,
       -0.08298343, -0.04912087, -0.01072922, -0.07058429,  0.07745163,
       -0.00535004, -0.06034842, -0.03129559,  0.106972  , -0.08280167,
        0.01221738,  0.00380703,  0.00080806, -0.03947047, -0.02006989])

In [109]:
atributivos = atributos[5809]

explain_model(forest_evaluation,atributivos,10,atributos)

array([-0.08181818,  0.01797348,  0.00432282, -0.01447259,  0.02898144,
        0.0449544 , -0.06832587, -0.01461441, -0.01552646, -0.06220822,
        0.02477694, -0.02245507, -0.02120837,  0.00905765, -0.05728057,
       -0.03692766, -0.01940025, -0.04822292, -0.0681671 ,  0.03582224,
       -0.03250363, -0.02845789, -0.07067799,  0.04551963, -0.06832587,
        0.00015877,  0.012292  ,  0.02249933,  0.0339091 , -0.03986767])

In [110]:
attri = attr[17378]

explain_model(red_bikes,attri,10,attr)

array([ -23.35786179, -103.90000024,  201.22922074,   76.74221986,
         40.34649733, -169.15943089,  139.32334684, -103.4536384 ,
         45.57438371,    5.35837132,  -61.8003497 ,  -48.4623393 ,
        151.79888326])

In [111]:
attri = attr[17378]

explain_model(forest_bikes,attri,10,attr)

array([ -27.54166575,  -95.45217128,  191.62074367,   63.82419624,
         49.62423809, -161.61241803,  146.8868667 , -112.79223177,
         20.6582665 ,  -18.95970989,  -52.3572794 ,  -65.40630694,
        128.64778787])

In [112]:
#En la definición de cada metrica se importarán las cosas necesarias por si solo se quiere probar una metrica
import numpy as np


def identidad(x1,x2,exp1,exp2):
    #Definición: El principio de identidad establece que objetos idénticos deben recibir explicaciones idénticas.

    res = True #Si los objetos no son identicos no hay que evaluar
    distance = np.linalg.norm(x1 - x2)#Calcula las distancias entre los dos ejemplos
    if(distance ==0):   #Solo comprobamos si las explicaciones sean identicas si los objetos son identicos
        res = np.linalg.norm(exp1 - exp2) ==0#Comprobamos si ambas explicaciones son iguales
    return res

In [113]:
#Hacemos una pequeña prueba para ver que la identidad funciona cogiendo dos veces el mimso ejemplo y viendo que se generan las 
#mismas explicaciones
ejemplo2=attr[17375]
ejemplo1 = attr[17375]
print(ejemplo2)
print(ejemplo1)
exp1 = explain_model(forest_bikes,ejemplo1,2,attr)
exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
print(identidad(ejemplo1,ejemplo2,exp1,exp2))

[0.         1.         1.         0.86956522 0.         0.16666667
 1.         0.33333333 0.24489796 0.2576     0.6        0.19301751
 0.02179837 0.09142212]
[0.         1.         1.         0.86956522 0.         0.16666667
 1.         0.33333333 0.24489796 0.2576     0.6        0.19301751
 0.02179837 0.09142212]
True


In [114]:
#Comprobaremos la métrica identidad
#Para comprobarla recorreremos 256 ejemplos en los cuatro modelos y para cada ejemplo en cada modelo generaremos 
#la explicacion dos veces y comprobaremos que es la misma explicación
atributivos = atributos[5553:5809,:] #seleccionamos varias muestras, si cambiamos el número de la izquierda de los dos puntos:, podemos hacer que se utilizen más o menos muestras
sonIguales = True #Inicializamos la varibale
for x in atributivos: #Recorremos las muestras
    exp1 = explain_model(forest_evaluation,x,2,atributos) #Generamos una explicación para una muestra
    exp2 = explain_model(forest_evaluation,x,2,atributos) #Volvemos a generar una explicación para la misma muestra
    sonIguales = sonIguales and identidad(x,x,exp1,exp2) #Comprobamos si cumplen la identidad
    if(not sonIguales): #Cuando no se cumpla rompemos el bucle
        break
#Mostrarmos el resualdo de evaluar la metrica identidad
if(sonIguales):
    print("El modelo RandomForest de evaluación cumple la métrica identidad")
else:
    print("El modelo RandomForest de evaluación no cumple la métrica identidad")

#Repetimos el proceso para los otros tres modelos, para los modelos de bike, además, utilizamos otros datos, como es normal
    
sonIguales=True
for x in atributivos:
    exp1 = explain_model(red_evaluation,x,2,atributos)
    exp2 = explain_model(red_evaluation,x,2,atributos)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break

if(sonIguales):
    print("El modelo Red neuronal de evaluación cumple la métrica identidad")
else:
    print("El modelo Red neuronal de evaluación no cumple la métrica identidad")

X=attr[17122:17378,:]
sonIguales=True
for x in X:
    exp1 = explain_model(red_bikes,x,2,attr)
    exp2 = explain_model(red_bikes,x,2,attr)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break
    
if(sonIguales):
    print("El modelo Red neuronal de bikes cumple la métrica identidad")
else:
    print("El modelo Red neuronal de bikes no cumple la métrica identidad")
    
sonIguales=True
for x in X:
    exp1 = explain_model(forest_bikes,x,2,attr)
    exp2 = explain_model(forest_bikes,x,2,attr)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break
    
if(sonIguales):
    print("El modelo RandomForest de bikes cumple la métrica identidad")
else:
    print("El modelo RandomForest de bikes no cumple la métrica identidad")

El modelo RandomForest de evaluación cumple la métrica identidad
El modelo Red neuronal de evaluación cumple la métrica identidad
El modelo Red neuronal de bikes cumple la métrica identidad
El modelo RandomForest de bikes cumple la métrica identidad


In [115]:
import numpy as np


def separabilidad(x1,x2,exp1,exp2):
#Definición:#Separabilidad: Objetos no idénticos no pueden tener explicaciones idénticas. Para simplificar, cada característica 
#tiene un nivel mínimo de importancia, positivo o negativo, en las predicciones.

    res = True#Iicializamos la variable
    distance = np.linalg.norm(x1 - x2)#Calcula las distancias entre los dos ejemplos
    if(distance >0):   #Si la distancia es distinta de cero comparamos las explicaciones
        res = not(np.linalg.norm(exp1 - exp2) ==0)#Si las explicaciones son distintas se cumple la separabilidad
    
    return res

In [116]:
#Hacemos una pequeña prueba para comprobar que la separabilidad funciona correctamente, comparando dos muestras distintas
#y comprobando que sus explicaciones también son distintas. Aunque se parezcan bastante
ejemplo2=attr[17376]
ejemplo1 = attr[17375]
print(ejemplo2)
print(ejemplo1)
exp1 = explain_model(forest_bikes,ejemplo1,2,attr)
exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
print(separabilidad(ejemplo1,ejemplo2,exp1,exp2))

[0.         1.         1.         0.91304348 0.         0.16666667
 1.         0.         0.24489796 0.2576     0.6        0.19301751
 0.01907357 0.09367946]
[0.         1.         1.         0.86956522 0.         0.16666667
 1.         0.33333333 0.24489796 0.2576     0.6        0.19301751
 0.02179837 0.09142212]
True


In [117]:
#Comprobaremos la métrica separabilidad
#Para comprobarla recorreremos 256 ejemplos en los cuatro modelos y los comparamos con otro ejemplo
ejemplo1=atributos[5552]#Escogemos una muestra para compararla con todas las muestras de un conjunto
atributivos = atributos[5553:5809,:] #Escogemos un conjunto de muestras 
cumpleSeparabilidad = True #Inicializamos una variable
for x in atributivos: #Recorremos las muestras
    exp1 = explain_model(forest_evaluation,x,2,atributos)#Generamos una explicación de la muestra que hemos escogido
    exp2 = explain_model(forest_evaluation,ejemplo1,2,atributos)#Y una explicación para cada muestra del conjunto
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo1,exp1,exp2) #Comprobamos si se cumple la separabilidad
    if(not cumpleSeparabilidad): #Si deja de cumplirse nos saltamos el bucle
        break
#Evaluamos si se cumple la separabilidad
if(cumpleSeparabilidad):
    print("El modelo RandomForest de evaluación cumple la métrica separabilidad")
else:
    print("El modelo RandomForest de evaluación no cumple la métrica separabilidad")

#Repetimos el proceso con los otros tres modelos     
    
cumpleSeparabilidad = True
for x in atributivos:
    exp1 = explain_model(red_evaluation,x,2,atributos)
    exp2 = explain_model(red_evaluation,ejemplo1,2,atributos)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo1,exp1,exp2)
    if(not cumpleSeparabilidad):
        break

if(cumpleSeparabilidad):
    print("El modelo Red neuronal de evaluación cumple la métrica separabilidad")
else:
    print("El modelo Red neuronal de evaluación no cumple la métrica separabilidad")

ejemplo2 = attr[17121]
X=attr[17122:17378,:]
cumpleSeparabilidad=True
for x in X:
    exp1 = explain_model(red_bikes,x,2,attr)
    exp2 = explain_model(red_bikes,ejemplo2,2,attr)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo2,exp1,exp2)
    if(not cumpleSeparabilidad):
        break
    
if(cumpleSeparabilidad):
    print("El modelo Red neuronal de bikes cumple la métrica separabilidad")
else:
    print("El modelo Red neuronal de bikes no cumple la métrica separabilidad")
    
cumpleSeparabilidad=True
for x in X:
    exp1 = explain_model(forest_bikes,x,2,attr)
    exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo2,exp1,exp2)
    if(not cumpleSeparabilidad):
        break
    
if(cumpleSeparabilidad):
    print("El modelo RandomForest de bikes cumple la métrica separabilidad")
else:
    print("El modelo RandomForest de bikes no cumple la métrica separabilidad")

El modelo RandomForest de evaluación cumple la métrica separabilidad
El modelo Red neuronal de evaluación cumple la métrica separabilidad
El modelo Red neuronal de bikes cumple la métrica separabilidad
El modelo RandomForest de bikes cumple la métrica separabilidad


In [118]:
import numpy as np
from scipy.stats import spearmanr

def estabilidad(x1,x2,exp1,exp2,umbEj,umbExp):
    #Objetos similares deben tener explicaciones similares
    #Para esto calcularemos la correlacion de Spearman para los ejemplos, si superan el limite que se pasará a la función
    #Calcularemos la correlación de Spearman para las explicaciones y comprobaremos si supera el umbral, en ese caso, cumpliran la estabilidad
    res = True #Inicializamos una variable
    correlacionEj, _ = spearmanr(x1, x2) #Calculamos la correlación de spearman de dos muestras
    if(abs(correlacionEj) >= umbEj): #Si es mayor que el umbral para las muestras, comprobaremos si sus explicaciones son parecidas
        correlacionExp, _ = spearmanr(exp1,exp2) #Calculamos la correlación de spearman de las explicaciones
        res = abs(correlacionExp)>=umbExp #Si es superior que el umbral de las explicaciones, se cumple la estabilidad
    return res

In [119]:
#Haremos una pequeña prueba para comprobar la que estabilidad funciona correctamente
#Escogemos dos muestras parecidas
ejemplo2=attr[17376]
ejemplo1 = attr[17375]
print(ejemplo2)
print(ejemplo1)
#Calulamos la correlacion de las muestras 
print(spearmanr(ejemplo1, ejemplo2))
#Generamos sus explicaciones
exp1 = explain_model(forest_bikes,ejemplo1,2,attr)
exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
#Calculamos la correlación de las explicaciones
print(spearmanr(exp1, exp2))
#Tras esto, observamos que si se cumple la estabilidad
print(estabilidad(ejemplo1,ejemplo2,exp1,exp2,0.75,0.75))#Utilizamos la función para ver que funciona correctamente

[0.         1.         1.         0.91304348 0.         0.16666667
 1.         0.         0.24489796 0.2576     0.6        0.19301751
 0.01907357 0.09367946]
[0.         1.         1.         0.86956522 0.         0.16666667
 1.         0.33333333 0.24489796 0.2576     0.6        0.19301751
 0.02179837 0.09142212]
SpearmanrResult(correlation=0.8770812523243259, pvalue=3.801615077865054e-05)
SpearmanrResult(correlation=1.0, pvalue=0.0)
True


In [120]:
#Comprobaremos la métrica estabilidad
#Para comprobarla recorreremos 256 ejemplos en los cuatro modelos y los comparamos con otro ejemplo, del cual sabemos que hay ejemplos muy similares

ejemplo1=atributos[5819]
atributivos = atributos[5553:5819,:]
cumpleEstabilidad = True
for x in atributivos:
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    exp2 = explain_model(forest_evaluation,ejemplo1,2,atributos)
    cumpleEstabilidad=cumpleEstabilidad and estabilidad(x,ejemplo1,exp1,exp2,0.75,0.75)
    if(not cumpleEstabilidad):
        break
if(cumpleEstabilidad):
    print("El modelo RandomForest de evaluación cumple la métrica separabilidad")
else:
    print("El modelo RandomForest de evaluación no cumple la métrica separabilidad")

cumpleEstabilidad = True
for x in atributivos:
    exp1 = explain_model(red_evaluation,x,2,atributos)
    exp2 = explain_model(red_evaluation,ejemplo1,2,atributos)
    cumpleEstabilidad=cumpleEstabilidad and estabilidad(x,ejemplo1,exp1,exp2,0.75,0.75)
    if(not cumpleEstabilidad):
        break

if(cumpleEstabilidad):
    print("El modelo Red neuronal de evaluación cumple la métrica separabilidad")
else:
    print("El modelo Red neuronal de evaluación no cumple la métrica separabilidad")

ejemplo2 = attr[17376]
X=attr[17122:17378,:]
cumpleEstabilidad=True
for x in X:
    exp1 = explain_model(red_bikes,x,2,attr)
    exp2 = explain_model(red_bikes,ejemplo2,2,attr)
    cumpleEstabilidad=cumpleEstabilidad and estabilidad(x,ejemplo2,exp1,exp2,0.75,0.75)
    if(not cumpleEstabilidad):
        break
    
if(cumpleEstabilidad):
    print("El modelo Red neuronal de bikes cumple la métrica estabilidad")
else:
    print("El modelo Red neuronal de bikes no cumple la métrica estabilidad")
    
cumpleEstabilidad=True
for x in X:
    exp1 = explain_model(forest_bikes,x,2,attr)
    exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
    cumpleEstabilidad=cumpleEstabilidad and estabilidad(x,ejemplo2,exp1,exp2,0.75,0.75)
    if(not cumpleEstabilidad):
        break
    
if(cumpleEstabilidad):
    print("El modelo RandomForest de bikes cumple la métrica estabilidad")
else:
    print("El modelo RandomForest de bikes no cumple la métrica estabilidad")

El modelo RandomForest de evaluación cumple la métrica separabilidad
El modelo Red neuronal de evaluación cumple la métrica separabilidad
El modelo Red neuronal de bikes cumple la métrica estabilidad
El modelo RandomForest de bikes cumple la métrica estabilidad


In [122]:
import numpy as np
from scipy import integrate
from scipy.integrate import trapz

def selectividad(x, model,exp):
    #Calcularemos el area bajo la cruva a partir de ir quitando las características más importantes
    y = x.copy() #Generamos una copia de la muestra recibida
    errores = [] #Aqui acumularemos los errores
    prediccionOriginal = 0 
    iteraciones= [] #Aqui guardaremos el número de iteraciones, necesario para calcular la integral
    if isinstance(model, keras.Sequential): #Dependiendo del modelo que reciba, necesita un tipo de datos u otro 
            prediccionOriginal = model.predict(x,verbose = 0) #Predecimos si es una red neuronal
    else:
            array = np.array(x)
            x = array.reshape(1, -1)
            prediccionOriginal = model.predict(x) #Si es random forest, utilizamos los datos de forma distinta
    
    for i in range(len(exp)):#Recorremos el número de parametros de las explicaciones
        masImportante = max(exp) #Busacmos la mayor importante
        atributo = np.where(exp == masImportante)[0] # Averiguamos en que posición se encuentra esta importancia
        y[atributo] = 0 #modificamos y la característica más importante en este momento
        exp[atributo] = -999999999999999999 #Cambiamos la importancia de esta característica haciendo que tenga un valor muy negativo para que nunca sea la más importante
         #Hacemos una predicción con la muestra modificada:
        if isinstance(model, keras.Sequential):
            z = model.predict(y,verbose=0)
            
        else:
            array = np.array(y)
            y2 = array.reshape(1, -1)
            z = model.predict(y2)
        iteraciones.append(i)
        errores.append(abs(prediccionOriginal-z))#Acumulamos el error
    
    #Cambiamos las dimensiones de ambos np.array, esto ocurre al meter las predicciones en una array, se modifica la dimensión de este
    errores = np.squeeze(errores) 
    res = trapz(iteraciones,errores) #Calculamos el area bajo la cruva
    return res


In [123]:
#Para probar la selectividad sobre el metodo lime, recorremos varias muestras y veremos cuales son sus selectividades y calcularemos una media
atributivos = atributos[5810:5819,:]#seleccionamos varias muestras, si cambiamos el número de la izquierda de los dos puntos:, podemos hacer que se utilizen más o menos muestras

selectividades = [] #Aqui acumularemos las selectividades

for i in range(len(atributivos)): #Recorremos las muestras
    exp = explain_model(red_evaluation,atributivos[i],2,atributos) #Generamos una explicación para cada muestra
    sel = selectividad(atributivos[i],red_evaluation,exp) #Calulamos la selectividad de esta muestra
    selectividades.append(sel)#Acumulamos la selectividades
    
print("Estas son las selectividades del modelo Red Evaluación ",selectividades) #Mostramos las selectividades
print("La selectividad media del modelo Red Evaluación es ", np.mean(selectividades)) #Mostramos la media

#Repetimos el proceso para los demás modelos

selectividades = []

for i in range(len(atributivos)):
    exp = explain_model(forest_evaluation,atributivos[i],2,atributos)
    sel = selectividad(atributivos[i],forest_evaluation,exp)
    selectividades.append(sel)
    
print("Estas son las selectividades del modelo RandomForest Evaluación ",selectividades)
print("La selectividad media del modelo RandomForest Evaluación es ", np.mean(selectividades))

X = attr[17360:17378,:]   

selectividades = []
for i in range(len(X)):
    exp = explain_model(red_bikes,X[i],2,attr)
    sel = selectividad(X[i],red_bikes,exp)
    selectividades.append(sel)
    
print("Estas son las selectividades del modelo Red bikes ",selectividades)
print("La selectividad media del modelo Red bikes es ", np.mean(selectividades))

selectividades = []

for i in range(len(X)):
    exp = explain_model(forest_bikes,X[i],2,attr)
    sel = selectividad(X[i],forest_bikes,exp)
    selectividades.append(sel)
    
print("Estas son las selectividades del modelo RandomForest bikes ",selectividades)
print("La selectividad media del modelo RandomForest bikes es ", np.mean(selectividades))


Estas son las selectividades del modelo Red Evaluación  [0.13354629278182983, 0.1803080439567566, 0.06457960605621338, 0.04284036159515381, -0.00476759672164917, 0.06457960605621338, 3.8172245025634766, 3.932610273361206, 0.05474388599395752]
La selectividad media del modelo Red Evaluación es  0.9206294417381287
Estas son las selectividades del modelo RandomForest Evaluación  [0.695, 0.6900000000000001, 0.0, 0.6899999999999998, 0.0, 0.0, 4.15, 4.170000000000001, 0.0]
La selectividad media del modelo RandomForest Evaluación es  1.155
Estas son las selectividades del modelo Red bikes  [-0.5851597785949707, 0.6832714080810547, 0.6243743896484375, 5.3707122802734375, 6.5587005615234375, 16.622493743896484, 20.209182739257812, 24.75194549560547, 18.846359252929688, 32.276954650878906, 35.121437072753906, 16.28521728515625, 6.051361083984375, 3.6080894470214844, 4.909355163574219, 3.0119285583496094, 3.1414031982421875, 7.246679306030273]
La selectividad media del modelo Red bikes es  11.374

In [129]:
def coherencia(mae1, mae2):
    #Se calcula la diferencia entre el error de predicción (mae1) sobre la señal original y el error de predicción mae2
    #de una nueva señal donde se eliminan las características no importantes.
    alpha_i = abs(mae1 - mae2)
    return alpha_i


In [128]:
#Para probar la coherencia lo que haremos será para varias muestras muestras, generar una predicción, generar su explicación 
#y a partir de su explicación quitaremos los N atributos menos importantes y generaremos otra explicación, a partir de ambas predicciones 
#calcularemos dos MAE y con ello calcularemos la coherencia
from sklearn.metrics import mean_absolute_error

#Escogemos un conjunto de musetras junto a sus valores objetivo
atributivos = atributos[5810:5819,:]
objetivos = objetivo[5810:5819]

N=2#Definimos el número de atributos que quitaremos
coherencias = []


for i in range(len(atributivos)):#Recorremos las muestras
    x = atributivos[i] 
    y = x.copy() #Generamos una copia de la muestra
    exp1 = explain_model(red_evaluation,x,2,atributos) #Generamos una explicación para la muestra
    prediccion1 = red_evaluation.predict(x,verbose=0) #Hacemos una predicción para la muestra
    errp = abs(objetivos[i]-prediccion1) #Calculamos el error entre la predicción y el objetivo
   
    for j in range(N): #Recorremos el número de atributos que queremos quitar
        menosImportante = min(exp1) #Cogemos la menor importancia
        atributo = np.where(exp1 == menosImportante)[0] #Buscamos a que atributo hace referencia
        y[atributo] = 0 #Quitamos el atributo
        exp1[atributo] = 99999999999999999 #Cambiamos el valor de esta imporatncia para que nunca más sea  la menor
    prediccion2 = red_evaluation.predict(y,verbose=0) #Generemos una predicción con la muestra modificada
   
    errexp = abs(objetivos[i]-prediccion2) #Calculamos el error de la muestra modificada
    res = coherencia(errp,errexp)
    coherencias.append(res) #Calculamos la coherencia y la metemos en su lista

#Cambiamos las dimensiones del array y los mostramos
coherencias = np.squeeze(coherencias)
print("El modelo Red Neuronal de evaluación tiene unas coherencias de ",coherencias)
print("El modelo Red Neuronal de evaluación tiene una coherencia media de de ",np.mean(coherencias))


#Repetimos el proceso con el resto de modelos

N=2
coherencias = []

for i in range(len(atributivos)):
    x = atributivos[i]
    y = x.copy()
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    x = x.reshape(1, -1)
    prediccion1 = forest_evaluation.predict(x)
    errp = abs(objetivos[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 999999999999999999
    y = y.reshape(1, -1)
    prediccion2 = forest_evaluation.predict(y)
      
    errexp = abs(objetivos[i]-prediccion2)
    res = coherencia(errp,errexp)
    coherencias.append(res) 
    
coherencias = np.squeeze(coherencias)
print("El modelo RandomForest de evaluación tiene unas coherencias de ",coherencias)
print("El modelo RandomForest de evaluación tiene una coherencia media de de ",np.mean(coherencias))


X=attr[17360:17378,:]
objs = obj[17360:17378]


N=2
coherencias = []


for i in range(len(X)):
    x = X[i]
    y = x.copy()
    exp1 = explain_model(red_bikes,x,2,attr)
    prediccion1 = red_bikes.predict(x,verbose=0)
    errp = abs(objs[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 99999999999999999
    prediccion2 = red_bikes.predict(y,verbose=0)
       
    errexp = abs(objs[i]-prediccion2)
    res = coherencia(errp,errexp)
    coherencias.append(res) 


coherencias = np.squeeze(coherencias)
print("El modelo Red Neuronal de bikes tiene unas coherencias de ",coherencias)
print("El modelo Red Neuronal de bikes tiene una coherencia media de de ",np.mean(coherencias))    


N=2
coherencias = []

for i in range(0,len(X)):
    x = X[i]
    y = x.copy()
    exp1 = explain_model(forest_bikes,x,2,attr)
    x = x.reshape(1, -1)
    prediccion1 = forest_bikes.predict(x)
    errp = abs(objs[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 999999999999999999
    y = y.reshape(1, -1)
    prediccion2 = forest_bikes.predict(y)
    errexp = abs(objs[i]-prediccion2)
    res = coherencia(errp,errexp)
    coherencias.append(res) 
    

    
coherencias = np.squeeze(coherencias)
print("El modelo RandomForest de bikes tiene unas coherencias de ",coherencias)
print("El modelo RandomForest de bikes tiene una coherencia media de de ",np.mean(coherencias))


El modelo Red Neuronal de evaluación tiene unas coherencias de  [4.2473960e-01 3.1312466e-01 1.2915921e-01 4.5259809e-01 2.8135777e-03
 1.2915921e-01 3.4266052e+00 3.5245893e+00 1.0948777e-01]
El modelo Red Neuronal de evaluación tiene una coherencia media de de  0.94580853
El modelo RandomForest de evaluación tiene unas coherencias de  [0.05 0.05 2.   0.05 1.38 2.   1.76 1.8  2.  ]
El modelo RandomForest de evaluación tiene una coherencia media de de  1.232222222222222
El modelo Red Neuronal de bikes tiene unas coherencias de  [ 30.308372     0.37038803  42.968124   148.86833    102.90701
  46.02864     75.51104    131.03801    122.52409    145.69594
 210.405      143.98051    108.95059     72.613365    67.9225
  41.204407    43.032455     6.1525726 ]
El modelo Red Neuronal de bikes tiene una coherencia media de de  85.582306
El modelo RandomForest de bikes tiene unas coherencias de  [3.1000e+01 2.0000e-02 4.3780e+01 1.5030e+02 1.0717e+02 4.6840e+01
 7.5880e+01 1.3578e+02 1.2880e+02 1

In [130]:
def completitud(ei, pi):
    #Evalúa el porcentaje de error de explicación con respecto al error de predicción.
    gamma_i = ei / pi
    return gamma_i

In [131]:
#Para evaluar la completitud, recorreremos varias muestras y calcularemos el error de prediccion y el de explicacion 
#para calcular la completitud
from sklearn.metrics import mean_absolute_error

#Escogemos varias muestras y sus objetivos
atributivos = atributos[5810:5819,:]
objetivos = objetivo[5810:5819]
#Necesitaremos además, los atributos sin normalizar
atributos2 = evaluation.loc[:, 'class':'Q27']
atributos2 = atributos2.to_numpy()
atributos2 = atributos2[5810:5819,:]
#Necesitaremos dos arrays para guardar las predicciones
predicciones1 = []
predicciones2 = []

for j in range(len(atributivos)):#Reccoremos las muestras
    x = atributivos[j]
    pred1 = red_evaluation.predict(x,verbose = 0) #Generamos una prediccion por muestra
    predicciones1.append(pred1)
    y = 0
    exp1 = explain_model(red_evaluation,x,2,atributos) #Generamos una prediccion por muestra
    atributo2 = atributos2[j]
    for i in range(len(exp1)): #Calculamos la prediccion de la esplicación, esto lo hacemos multiplicando para 
        y += exp1[i]*atributo2[i]#cada atributo, el atributo sin normalizar por la importancia de este atributo
    predicciones2.append(y)

predicciones1 = np.squeeze(predicciones1)  
predicciones2 = np.squeeze(predicciones2) 
mae1 = mean_absolute_error(objetivos, predicciones1)#Calculamos el error de predicción
mae2 = mean_absolute_error(objetivos, predicciones2)#Calculamos el error de explicaciones

res =  completitud(mae2,mae1)#Calculamos la completitud
#Mostramos la completitud
print("El modelo Red Neuronal de evaluación tiene unas completitudes de ",res)

#Repetimos el proceso para el resto de modelos

predicciones1 = []
predicciones2 = []
for j in range(len(atributivos)):
    x = atributivos[j]
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    x = x.reshape(1,-1)
    pred1 = forest_evaluation.predict(x)
    predicciones1.append(pred1)
    y = 0
    atributo2 = atributos2[j]
    for i in range(len(exp1)):
        y += exp1[i]*atributo2[i]
    predicciones2.append(y)

predicciones1 = np.squeeze(predicciones1)  
predicciones2 = np.squeeze(predicciones2) 
mae1 = mean_absolute_error(objetivos, predicciones1)
mae2 = mean_absolute_error(objetivos, predicciones2)

res =  completitud(mae2,mae1)
print("El modelo RandomForest de evaluación tiene una completitud de ",res)


X = attr[17360:17378,:]
objs = obj[17360:17378]

attr2 = bikes.loc[:, 'season':'registered']
attr2 = attr2.to_numpy()
attr2 = attr2[17360:17378,:]
predicciones1 = []
predicciones2 = []
for j in range(len(X)):
    x = X[j]
    pred1 = red_bikes.predict(x,verbose = 0)
    predicciones1.append(pred1)
    y = 0
    exp1 = explain_model(red_bikes,x,2,attr)
    atributo2 = attr2[j]
    for i in range(len(exp1)):
        y += exp1[i]*atributo2[i]
    predicciones2.append(y)

predicciones1 = np.squeeze(predicciones1)  
predicciones2 = np.squeeze(predicciones2) 
mae1 = mean_absolute_error(objs, predicciones1)
mae2 = mean_absolute_error(objs, predicciones2)

res =  completitud(mae2,mae1)
print("El modelo Red Neuronal de bikes tiene una completitud de ",res)

predicciones1 = []
predicciones2 = []
for j in range(len(X)):
    x = X[j]
    exp1 = explain_model(forest_bikes,x,2,atributos)
    x = x.reshape(1,-1)
    pred1 = forest_bikes.predict(x)
    predicciones1.append(pred1)
    y = 0
    atributo2 = attr2[j]
    for i in range(len(exp1)):
        y += exp1[i]*atributo2[i]
    predicciones2.append(y)

predicciones1 = np.squeeze(predicciones1)  
predicciones2 = np.squeeze(predicciones2) 
mae1 = mean_absolute_error(objs, predicciones1)
mae2 = mean_absolute_error(objs, predicciones2)

res =  completitud(mae2,mae1)
print("El modelo RandomForest de bikes tiene una completitud de ",res)
    

El modelo Red Neuronal de evaluación tiene unas completitudes de  22.22707616706536
El modelo RandomForest de evaluación tiene una completitud de  102.26523707710479
El modelo Red Neuronal de bikes tiene una completitud de  619.6128698731212
El modelo RandomForest de bikes tiene una completitud de  871.6249811588647


In [133]:
import numpy as np

def congruencia(coherencias):
    #Esta métrica ayuda a capturar la variabilidad de la coherencia.
    coherencia_media = np.mean(coherencias) #Calculamos la coherencia media
    res = (sum((c - coherencia_media)**2 for c in coherencias)/len(coherencias))**0.5 #A partir de ella, calculamos la variabilidad
    return res

In [134]:
#Para comprobar la congruencia, calcularemos la coherencia de varias muestras y apartir de eso se calculará la congruencia
from sklearn.metrics import mean_absolute_error
import numpy as np

#Cogemos varias meustras y sus valores objetivos
atributivos = atributos[5810:5819,:]
objetivos = objetivo[5810:5819]

#Defnimos el número de características a eliminar
N=2
coherencias = []

for i in range(len(atributivos)): #Recorremos las muestras
    x = atributivos[i]
    y = x.copy()
    exp1 = explain_model(red_evaluation,x,2,atributos) #Generamos una explicacion para cada muestra
    prediccion1 = red_evaluation.predict(x,verbose=0) #Hacemos una predicción
    errp = abs(objetivos[i]-prediccion1) #Calculamos el error de predicción
   
    for j in range(N): #Eliminaremos N características
        menosImportante = min(exp1) #Cogemos la menor importancia
        atributo = np.where(exp1 == menosImportante)[0] #Vemos a que atributo pertenece
        y[atributo] = 0 #Ponemos ese atributo a cero
        exp1[atributo] = 99999999999999999 #Cambiamos la importancia de esta característica para que nunca más sea la mínima
    prediccion2 = red_evaluation.predict(y,verbose=0) #Hacemos una predicción con la muestra modificada
   
    errexp = abs(objetivos[i]-prediccion2) #Calculamos el error de la predicción de la muestra modificada
    coherencias.append(coherencia(errp,errexp))
    

coherencias = list(coherencias)
congr = congruencia(coherencias) #Calculamos la congruencia

    
#Mostramos la congruencia
print("El modelo Red Neuronal de evaluación tiene una congruencia de ",congr)

#Repetimos el proceso para los demás modelos

N=2
coherencias = []

for i in range(len(atributivos)):
    x = atributivos[i]
    y = x.copy()
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    x = x.reshape(1, -1)
    prediccion1 = forest_evaluation.predict(x)
    errp = abs(objetivos[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 999999999999999999
    y = y.reshape(1, -1)
    prediccion2 = forest_evaluation.predict(y)
      
    errexp = abs(objetivos[i]-prediccion2)
    coherencias.append(coherencia(errp,errexp))
    
coherencias = list(coherencias)
congr = congruencia(coherencias)

print("El modelo RandomForest de evaluación tiene una congruencia de ",congr)

X=attr[17360:17378,:]
objs = obj[17360:17378]


N=2
coherencias = []


for i in range(len(X)):
    x = X[i]
    y = x.copy()
    exp1 = explain_model(red_bikes,x,2,attr)
    prediccion1 = red_bikes.predict(x,verbose=0)
    errp = abs(objs[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 99999999999999999
    prediccion2 = red_bikes.predict(y,verbose=0)
       
    errexp = abs(objs[i]-prediccion2)
    coherencias.append(coherencia(errp,errexp))

coherencias = list(coherencias)
congr = congruencia(coherencias)    

print("El modelo Red Neuronal de bikes tiene una congruencia de ",congr)


N=2
coherencias = []

for i in range(0,len(X)):
    x = X[i]
    y = x.copy()
    exp1 = explain_model(forest_bikes,x,2,attr)
    x = x.reshape(1, -1)
    prediccion1 = forest_bikes.predict(x)
    errp = abs(objs[i]-prediccion1)
    
   
    for i in range(N):
        menosImportante = min(exp1)
        atributo = np.where(exp1 == menosImportante)[0]
        y[atributo] = 0
        exp1[atributo] = 999999999999999999
    y = y.reshape(1, -1)
    prediccion2 = forest_bikes.predict(y)
    errexp = abs(objs[i]-prediccion2)
    coherencias.append(coherencia(errp,errexp))
    
coherencias = list(coherencias)
congr = congruencia(coherencias)

print("El modelo RandomForest de bikes tiene una congruencia de ",congr)


El modelo Red Neuronal de evaluación tiene una congruencia de  [[1.3598213]]
El modelo RandomForest de evaluación tiene una congruencia de  [0.85532031]
El modelo Red Neuronal de bikes tiene una congruencia de  [[55.404655]]
El modelo RandomForest de bikes tiene una congruencia de  [55.63925418]
